In [ ]:
pip install tensorflow keras scikit-learn flask opencv-python matplotlib


In [1]:
import os
import zipfile
import shutil
import numpy as np
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications import MobileNetV2
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D, Dropout
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping
import joblib

# Path to dataset ZIP
ZIP_FILE_PATH = "/content/ZIDIO_DATA_FINAL.zip"
# Corrected UNZIPPED_DIR to point to the extracted directory
UNZIPPED_DIR = "/content/ZIDIO_DATA_FINAL"

# Unzip if not already done
if not os.path.exists(UNZIPPED_DIR):
    os.makedirs(UNZIPPED_DIR)
    with zipfile.ZipFile(ZIP_FILE_PATH, 'r') as zip_ref:
        zip_ref.extractall(UNZIPPED_DIR)

# Constants
IMAGE_SIZE = 224
BATCH_SIZE = 32
EPOCHS = 20  # Increased for better learning
LEARNING_RATE = 1e-4

# Augmentation + validation split
datagen = ImageDataGenerator(
    rescale=1./255,
    validation_split=0.2,
    horizontal_flip=True,
    rotation_range=20,
    zoom_range=0.2
)

train_data = datagen.flow_from_directory(
    UNZIPPED_DIR,
    target_size=(IMAGE_SIZE, IMAGE_SIZE),
    batch_size=BATCH_SIZE,
    class_mode='sparse',
    subset='training'
)

val_data = datagen.flow_from_directory(
    UNZIPPED_DIR,
    target_size=(IMAGE_SIZE, IMAGE_SIZE),
    batch_size=BATCH_SIZE,
    class_mode='sparse',
    subset='validation'
)

# Load MobileNetV2 base model
base_model = MobileNetV2(weights='imagenet', include_top=False, input_shape=(IMAGE_SIZE, IMAGE_SIZE, 3))
base_model.trainable = False  # Freeze base initially

# Add custom layers
x = base_model.output
x = GlobalAveragePooling2D()(x)
x = Dropout(0.5)(x)  # Regularization
x = Dense(128, activation='relu')(x)
predictions = Dense(train_data.num_classes, activation='softmax')(x)

model = Model(inputs=base_model.input, outputs=predictions)

model.compile(optimizer=Adam(learning_rate=LEARNING_RATE), loss='sparse_categorical_crossentropy', metrics=['accuracy'])

# Callbacks
early_stop = EarlyStopping(monitor='val_loss', patience=3, restore_best_weights=True)

# Train
model.fit(train_data, epochs=EPOCHS, validation_data=val_data, callbacks=[early_stop])

# Fine-tune base model (optional)
base_model.trainable = True
model.compile(optimizer=Adam(learning_rate=1e-5), loss='sparse_categorical_crossentropy', metrics=['accuracy'])
model.fit(train_data, epochs=5, validation_data=val_data, callbacks=[early_stop])

# Save model & labels
os.makedirs('../backend/model', exist_ok=True)
model.save('../backend/model/emotion_model.h5')
joblib.dump(train_data.class_indices, '../backend/model/labels.pkl')




Found 122 images belonging to 3 classes.
Found 29 images belonging to 3 classes.
9406464/9406464 ━━━━━━━━━━━━━━━━━━━━ 2s 0us/step


/usr/local/lib/python3.11/dist-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


Epoch 1/20
4/4 ━━━━━━━━━━━━━━━━━━━━ 50s 10s/step - accuracy: 0.2938 - loss: 1.6581 - val_accuracy: 0.3793 - val_loss: 1.2434
Epoch 2/20
4/4 ━━━━━━━━━━━━━━━━━━━━ 21s 5s/step - accuracy: 0.2969 - loss: 1.4602 - val_accuracy: 0.3793 - val_loss: 1.1894
Epoch 3/20
4/4 ━━━━━━━━━━━━━━━━━━━━ 23s 5s/step - accuracy: 0.3775 - loss: 1.1687 - val_accuracy: 0.3103 - val_loss: 1.2020
Epoch 4/20
4/4 ━━━━━━━━━━━━━━━━━━━━ 22s 6s/step - accuracy: 0.4981 - loss: 1.0954 - val_accuracy: 0.3103 - val_loss: 1.2888
Epoch 5/20
4/4 ━━━━━━━━━━━━━━━━━━━━ 25s 7s/step - accuracy: 0.4042 - loss: 1.2201 - val_accuracy: 0.4483 - val_loss: 1.2694
Epoch 1/5
4/4 ━━━━━━━━━━━━━━━━━━━━ 107s 14s/step - accuracy: 0.2225 - loss: 1.4083 - val_accuracy: 0.2759 - val_loss: 1.2624
Epoch 2/5
4/4 ━━━━━━━━━━━━━━━━━━━━ 22s 5s/step - accuracy: 0.3578 - loss: 1.3184 - val_accuracy: 0.3103 - val_loss: 1.1906
Epoch 3/5
4/4 ━━━━━━━━━━━━━━━━━━━━ 22s 6s/step - accuracy: 0.3673 - loss: 1.1817 - val_accuracy: 0.2414 - val_loss: 1.2320
Epoch 4/

['../backend/model/labels.pkl']